## 5. Kuberentes Pods

### 5.1 파드(Pods) 란?
- 쿠버네티스가 배포하는 최소의 단위
- 파드라는 단위로 컨테이너를 묶어서 관리
    - 단일 컨테이너를 관리하는 것이 아닌 파드로 묶어서 관리!
    - ex) 외부에서 파드에 접근시 192.168.x.x라는 IP로 접근, 컨테이너와 통신시 컨테이너마다 다르게 설정한 포트로 접근
- 여러 개의 컨테이너가 하나의 Pod를 통해서 배포되었을때, localhost를 통해서 통신이 가능
- Pod 내에 배포된 컨테이너간에는 디스크 볼륨을 공유 가능
    - 애플리케이션이 실행 될 때 Reverse proxy, 로그 수집기등 다양한 주변 솔루션이 같이 배포 되는 경우가 많음
    - 쿠버네티스의 경우 하나의 Pod 내에서는 컨테이너들끼리 볼륨을 공유할 수 있어 로그 수집기가 애플리케이션의 로그 파일 수집 가능 

![kubenetes_pod_volume](img/kubenetes_pod_volume.png)

- 파드 내 컨테이너 형태 존재
    - 기동 시점에 처리하고 종료되고 Init Container
    - 실제 업무를 처리하는 Runtime Container (App Container)
    - 보조 역할로써 배포되는 SideCar Container
        - 애플리케이션 컨테이너와 주변 프로그램을 같이 배포하는 것
        - Ambassador, Adapter Container 등 여러 패턴 존재
    
<hr>



### 5.2 파드의 생명주기
- Pending : 쿠버네티스 시스템에 파드를 생성되는 중
    - 이미지 다운 받는 중 / 파드 내부 컨테이너 실행 중.
- Running : 파드 안 모든 컨테이너가 실행중인 상태
- Succeeded : 파드 안 모든 컨테이너가 정상 실행 된 상태
- Failed : 파드 안 컨테이너 중 정상적으로 실행 종료되지 않은 컨테이너
- Unknown : 파드의 상태를 확인 할 수 없는 상태 
    - 일반적으로 해당 노드와 통신 불가능 할 때, 연락이 끊겼거나... 등

<hr>


### 5.3 Kubelet으로 컨테이너 진단 (self-healing의 핵심)
- 컨테이너가 실행 된 후 **kubelet이 컨테이너 주기적인 진단 및 상태 확인**
- 컨테이너가 실행됬는지 확인, 실패시 kubelet이 컨테이너 종료 후, 재시작
- Pod가 재시작되도 IP는 변동되지 않으며, Pod가 계속 실행 할 수 있음을 보장
- Probe란 컨테이너에서 kubelet에 의해 주기적으로 수행되는 진단

- 방법 1) 활성프로브(livenessProbe) 
    - 컨테이너가 동작 중인지 여부 체크
    - 실패시 컨테이너 죽이고 재시작
    - ex) 컨테이너가 이슈나 프로세스 문제로 중단될 경우, 컨테이너가 종료되거나 재시작 되기를 원할 때 (좀 refresh 한느낌)
    
- 방법 2) 준비성프로브(readinessProbe)
    - 컨테이너가 서비스 요청에 응답을 할 수 있는 상태인지 체크
    - 상태확인 실패시, 해당 포드가 연결된 모든 서비스의 엔드포인트 정보 제거
    - ex) 해당 파드에 제대로 트래픽 전송을 하려는 경우 (대량의 데이터, 설정 파일등..)
- 방법 3) startupProbe
    - 컨테이너 내의 애플리케이션이 시작되었는지 체크
    
    
- 실제 핸들러
    - httpGet probe
        - 지정한 IP주소, port, path에 HTTP Get 요청을 보내 확인
        - 해당 Container의 응답 확인 반환코드가 200이 아닌 값이 나오면 컨테이너 재시작 
        - 필드 : httpGet: / path: / port:
    - tcpSocket probe
        - 지정된 포트에 TCP 연결을 시도
        - 연결되지 않으면 컨테이너 다시 시작
        - 필드 : tcpSocket: / port: 
    - exec probe
        - exec 명령을 전달 (exec : command -ls )
        - 명령의 종료코드가 0이 아니면 컨테이너 재시작

- 기타 필드 (부가 조건 옵션)
    - initialDelaySeconds : 최초 검사 실행 전 대기
    - periodSeconds : x초 interval로 검사
    - timeoutseconds
    - successThreshold : 성공 횟수 기준
    - failureThreshold : 실패 횟수 기준
    
![kubernetes_pod_livenessprobe](img/kubernetes_pod_livenessprobe.png)

<hr>

## 5.4 init Container
- init continaer(db or network, 파일권한)등의 사전 환경셋팅이 완료됬는지 체크 목적
- init/main container를 같은 Pod로 묶어놓고, init가 완료되면 main 작동
- 초기화 컨테이너가 모두 실행된 후에 앱 컨테이너를 실행
- 보안상 이유로 앱 컨테이너 이미지에 포함하지 않는 유틸리티 및 정보 

- 실제 아래 이미지에서 init-myservice와 init-mydb container가 다 실행된 후 myapp-containers가 실행됨

![kubenetes_initcontainer0](img/kubenetes_initcontainer0.PNG)

![kubenetes_initcontainer1](img/kubenetes_initcontainer1.PNG)

<hr>

### 5.5 infra container (= pause container)
- 모든 파드에서 항상 실행되는 pause 컨테이너
- 파드에 대한 infra (IP, Hostname) 등을 관리 해주는 container
- **Pods 내에서 PID 1을 가지고 실행되어 다른 컨테이너에 대한 부모 컨테이너 역할**
- pod의 환경을 만들어주는 container
- 파드 안 다른 컨테이너가 재시작되면 파드의 IP를 유지하는게 일반적,
- Pause 컨테이너가 재시작되면 파드 안 모든 컨테이너 재시작

![kubenetes_pod_infra_container](img/kubenetes_pod_infra_container.png)

<hr>

## 5.6 Pod에 리소스(CPU 연산량, 메모리 자원) 할당
- 노드 하나에 여러개 파드가 몰리게 되면, 파드들의 성능이 나빠지고, 자원 사용 효율이 낮음.
- 각 컨테이너가 CPU나 메모리를 얼마나 사용할 수 있을지 조건을 지정하는 것
- 필드 종류
    - Requests
        - 파드를 실행하기 위한 최소 리소스 양을 요청
    - Limits
        - 파드가 사용할 수 있는 최대 리소스 양을 제한
        - Memory limit을 초과해서 사용되는 파드는 종료(OOM Kill)되며 다시 스케쥴링 됨
    - 필드
        - .spec.containers[].resources.limits.cpu
        - .spec.containers[].resources.limits.memory
        - .spec.containers[].resources.requests.cpu
        - .spec.containers[].resources.requests.memory

![kubernetes_pod_resource](img/kubernetes_pod_resource.PNG)

<hr>

### 5.7 Pod 환경 변수
- Pod내의 컨테이너가 실행될 때 필요로 하는 변수
- 컨테이너 제작시 미리 정의된 컨테이너 환경변수를 변경 가능
    - ENV NGINX_VERSION 1.19.2 / ENV NJS_VERSION 0.4.3
- 사용할 env 환경 변수의 이름을 설정(name)
- 문자열이나 숫자 형식의 값을 설정 (value)

![kubernetes_pod_env](img/kubernetes_pod_env.PNG)

<hr>

### 5.8 Pod 구성 패턴 

- 사이드카(Sidecar) 패턴
    - 기본 컨테이너는 원래 목적의 기능에만 충실하고, 나머지 공통 부가 기능들은 사이드카 컨테이너 추가해서 사용
    - ex) 웹 서버 컨테이너와 로그 수집 컨테이너, 웹 서버 컨테이너를 다른 것으로 바꿔도 로그 컨테이너 그대로 수행 가능

- Adapter 패턴
    - 파드 외부로 노출되는 정보를 표준화
    - 어댑터 컨테이너로 파드의 모니터링 지표를 app-container를 통해 표준화한 형식으로 외부에 노출

- Ambassador (엠버서더)
    - 웹서버가 만든것을 받아서 내부에서 가공해서 Production들에 보내 주는 것
    - 앰배서더 패턴은 파드 안에서 프록시 역할을 하는 컨테이너를 추가하는 패턴


## 참고문헌
- https://medium.com/finda-tech/kubernetes-pod%EC%9D%98-%EC%A7%84%EB%8B%A8%EC%9D%84-%EB%8B%B4%EB%8B%B9%ED%95%98%EB%8A%94-%EC%84%9C%EB%B9%84%EC%8A%A4-probe-7872cec9e568
- https://coffeewhale.com/k8s/network/2019/04/19/k8s-network-01/